In [ ]:
import re
import string
import numpy as np
import Levenshtein as lv # ratio(), jaro(), jaro_winkler() for short strings
import pandas as pd
import pyodbc # for Hadoop
import json # for Hadoop
import time
import matplotlib.pyplot as plt
import seaborn as sns 
import os

from jupyterthemes import jtplot
jtplot.style(theme="monokai", context="notebook", ticks=True, grid=False)

with open('HIVE_JSON') as json_file:     # if you receive error here
    HIVE_JSON = json.load(json_file);    # you need to delete these two rows
    
HIVE_User = HIVE_JSON['username']        # insert your user credentials after '=' symbol
HIVE_Password = HIVE_JSON['password']    # insert your password credentials after '=' symbol

script_start = time.time()

# Functions

In [ ]:
def show(dataframe, x):
    with pd.option_context("display.max_rows",None, "display.max_columns",None):
        display(dataframe.head(x))

# Hadoop connection to Customer Master

In [ ]:
Country_mapping = pd.DataFrame(pd.read_excel('Country ISO.xlsx', sheet_name = 'ISO Country'))

Country_mapping.info()
Country_mapping.head(5)

In [ ]:
%%time

cnxnstr = """Driver={Hortonworks Hive ODBC Driver};HIVESERVERTYPE=2;HOST=sdeu5002.ux.festo.net;PORT=10000;
                 UID=""" + str(HIVE_User) + ';PWD=' + str(HIVE_Password) + ';AuthMECH=4;TrustedCerts=cacerts.pem'

cnxn = pyodbc.connect(cnxnstr, autocommit = True)

sql = """ SELECT customer, name, street, city, postal_cd, country
            FROM snac.t_bd_p08_y08hcv_md_rep_cust_sales_hadoop
            WHERE division = "01" and distr_chan = '01' and country = 'DE'
            ;"""   

Customer_Master = pd.DataFrame(pd.read_sql(sql, cnxn))

Customer_Master.info()
Customer_Master.head(10)

In [ ]:
# %%time
# sql = """ SELECT *
#             FROM snac.t_bd_p08_y08hcv_md_rep_cust_sales_hadoop
#             LIMIT 10;"""   

# show(pd.DataFrame(pd.read_sql(sql, cnxn)), 5)

In [ ]:
# print(Customer_Master['postal_cd'].str.split('-').str[0].str.len().mean().round())
# print(Customer_Master['postal_cd'].str.len().mean().round())

# # Customer_Master['postal_cd'] = Customer_Master['postal_cd'].str.split('-').str[0]
# Customer_Master['postal_cd'].head(5)

# Bisnode Data Loading

In [ ]:
Bisnode = pd.DataFrame(pd.read_excel(r'C:\Users\lt0grmk\OneDrive - Festo\Big Data\Company Matcher\Germany\Festo-1784-DE-2020-05.xlsx', 
                                      sheet_name = 'Tabelle1'))

Bisnode.info()
Bisnode.head(5)

In [ ]:
relevant_columns = ['name', 'street', 'city', 'postal_cd', 'country', 'customer']
relevant_columns

In [ ]:
Bisnode.rename(columns={"add": "street", "post code": "postal_cd"}, inplace = True) # make sure both DF's column names match
Bisnode['country'].value_counts(normalize = True).plot.bar(figsize = (20, 7));

In [ ]:
%%time
cm_country_count = len(Customer_Master['country'].value_counts().keys())
bisnode_country_count = len(Bisnode['country'].value_counts().keys())

Customer_Master = Customer_Master.merge(Country_mapping[['COUNTRY', 'A2 (ISO)']],
                                    left_on = "country", right_on = 'A2 (ISO)', how = 'inner').groupby(['customer']).head(1)

Customer_Master.drop(columns = ['A2 (ISO)', 'country'], inplace = True)
Customer_Master.rename(columns = {"COUNTRY": "country"}, inplace = True)

if bisnode_country_count != cm_country_count:
    
    Customer_Master = Customer_Master.merge(Bisnode['country'], on ='country', how = 'inner').groupby(['customer']).head(1)
    Customer_Master = Customer_Master[~Customer_Master.customer.isna()]

print("Shape: ", Customer_Master.shape)

Customer_Master.head(5)

In [ ]:
# # Bisnode['postal_cd'].astype(str).str[:5]
# # Bisnode['postal_cd'] = Bisnode['postal_cd'].astype(str).str[:5]

# print(Bisnode['postal_cd'].astype(str).str.split('-').str[0].str.len().mean().round())
# print(Bisnode['postal_cd'].astype(str).str.len().mean().round())

# # Bisnode['postal_cd'] = Bisnode['postal_cd'].str.split('-').str[0]
# Bisnode['postal_cd'].head(5)

# DataFrame prep

In [ ]:
Cust_Master_2 = Customer_Master[relevant_columns].copy()
Cust_Master_2 = Cust_Master_2[~Cust_Master_2[relevant_columns].duplicated()]

for col in Cust_Master_2.keys():
    if col != "customer":
        Cust_Master_2[col + "_org"] = Cust_Master_2[col]

Cust_Master_2.info()
Cust_Master_2.head(5)

In [ ]:
relevant_columns_bis = relevant_columns.copy()
relevant_columns_bis.remove('customer')
relevant_columns_bis.append('duns no')

Bisnode_org = Bisnode[relevant_columns_bis].copy()
# Bisnode_org = Bisnode_org.sample(n = 10, random_state = 42) # limit here n Bisnode Customer amount for testing

Bisnode_org.info()
Bisnode_org.head(5)

In [ ]:
%%time

symbol_lst = [")", "(", ".", "|", "[", "]", "{", "}", "'", "-", "!", "@", "#", "_", "%", "&", "$", '"', ".", ",", ':', ';', '=', '/', '?', '+']
rx = '[' + re.escape(''.join(symbol_lst)) + ']'
    
def str_clean(df):
    for column in df.columns:
        if column[-3:] != 'org' and column != 'duns no':
            df[column] = [re.sub(rx, "", str(x)) for x in df[column]]
            df[column] = df[column].str.replace(" ","")
            df[column] = df[column].str.lower()
            
str_clean(Cust_Master_2)
str_clean(Bisnode_org)

Bisnode_org.head(5)

In [ ]:
Cust_Master_2.head(5)

In [ ]:
print("Shape: ", Cust_Master_2.shape)
Cust_Master_2 = Cust_Master_2[~(Cust_Master_2.name == ".") |
                                ~(Cust_Master_2['name'].str.lower() == 'test') |
                                ~(Cust_Master_2['name'].str.lower() == '-') |
                                ~(Cust_Master_2['name'].str.lower() == '_') |
                                ~(Cust_Master_2['name'] == ':') |
                                ~(Cust_Master_2['name'] == '') |
                                ~(Cust_Master_2['name'].str.lower().str.contains('duplicate')) |
                                ~(Cust_Master_2['name'].str.lower().str.contains('unassigned'))]

Cust_Master_2 = Cust_Master_2[~(Cust_Master_2['city'] == "") & ~(Cust_Master_2['street'] == "")]
Cust_Master_2 = Cust_Master_2[~(Cust_Master_2['city'] == "-") & ~(Cust_Master_2['street'] == "-")]

print("Shape: ", Cust_Master_2.shape)

# String Comparison

In [ ]:
Bisnode_org.shape, Cust_Master_2.shape

In [ ]:
relevant_columns.remove('country')
relevant_columns.remove('customer')

bis_col = relevant_columns.copy()
bis_col.append('key')
bis_col.append('duns no')

cm_col = relevant_columns.copy()
cm_col.append('key')
cm_col.append('customer')

In [ ]:
%%time 
# cartesian product
print("Count of Countries: ", bisnode_country_count)

n_Neighbours = 1 # change here for neighbour count
name_weight, street_weight, city_weight, postal_cd_weight = 4, 3, 1, 1
Bisnode_org['key'], Cust_Master_2['key'] = 1, 1

dict_of_df = {}

for i in Bisnode_org['country'].value_counts().keys():
        dict_of_df[i] = pd.merge(Bisnode_org[bis_col][Bisnode_org.country == i].astype('category'),  Cust_Master_2[cm_col][Cust_Master_2.country == i].astype('category'),
                             on = 'key', suffixes = ("_Bisnode", "_Cust_Master"), copy = False).drop(columns = ['key'])
        
        for col in relevant_columns:
            dict_of_df[i][str(col) + '_score'] = list(map(lv.ratio, dict_of_df[i][str(col) + '_Bisnode'].to_numpy(), dict_of_df[i][str(col) + '_Cust_Master'].to_numpy()))
            dict_of_df[i][str(col) + '_score'] = dict_of_df[i][str(col) + '_score'].astype(np.float16)
    
#         dict_of_df[i]['final_score'] = dict_of_df[i].filter(regex = '_score$', axis = 1).mean(axis = 1) # mean calculation of all 'score' columns
        dict_of_df[i]['final_score'] = ((dict_of_df[i]['name_score'].to_numpy() * name_weight) + (dict_of_df[i]['street_score'].to_numpy() * street_weight) +
                                        (dict_of_df[i]['postal_cd_score'].to_numpy() * postal_cd_weight) + (dict_of_df[i]['city_score'].to_numpy() * city_weight)) / (name_weight + street_weight + city_weight + postal_cd_weight)
       
        dict_of_df[i]['final_score'] = dict_of_df[i]['final_score'].astype(np.float16)

        dict_of_df[i] = dict_of_df[i][dict_of_df[i]['final_score'].to_numpy() > 0]          
        dict_of_df[i] = dict_of_df[i].sort_values(['final_score'], ascending = False).groupby(['duns no']).head(n_Neighbours)

In [ ]:
%%time
df = []
df = pd.concat([dict_of_df[i] for i in [*dict_of_df]], axis = 0)

for i in [*dict_of_df]:
    try:
        del dict_of_df[i]
    except:
        pass

df.info()
df.head(5)

In [ ]:
for col in df.columns:
        col_type = df[col].dtype
        if col_type == 'float16':
            df[col] = df[col].astype(np.float64)

In [ ]:
df['final_score'].plot.hist(figsize = (15, 6), bins = 10);
pd.concat([df['final_score'].round(1).value_counts(normalize = False), df['final_score'].round(1).value_counts(normalize = True).round(2)], axis=1).head(10)

In [ ]:
score_threshold = .8

print("Count of rows that meets threshold:", len(df[df['final_score'] >= score_threshold]))
print("Share of rows that meets threshold:", round(len(df[df['final_score'] >= score_threshold]) / len(df), 2))

show(df[df['final_score'] >= score_threshold].sort_values(['name_Bisnode', 'final_score']), 5)

In [ ]:
print("Shape: ", df[df[['name_Bisnode', 'street_Bisnode', 'city_Bisnode']].duplicated()].shape)
df[df[['name_Bisnode', 'street_Bisnode', 'city_Bisnode']].duplicated()].head(10)

# Final Result Export

In [ ]:
df = df.merge(Cust_Master_2[['name_org', 'street_org', 'city_org', 'postal_cd_org', 'customer']], on = 'customer', how = 'inner')

In [ ]:
final_df = df[['duns no', 'customer', 'final_score', 'name_org', 'street_org', 'city_org', 'postal_cd_org']]
final_df.head(5)

In [ ]:
# Cust_Master_2[Cust_Master_2['customer'] == '0000727960']

In [ ]:
# Bisnode_org[Bisnode_org['duns no'] == '328897137']

In [ ]:
final_df.to_excel(r"C:\Users\lt0grmk\OneDrive - Festo\Big Data\Company Matcher\Germany\F-DE - Customer Matching Output.xlsx",
             sheet_name='Sheet1', index = False)  
print("Excel file saved!")

In [ ]:
script_end = time.time() 
time_delta = round(script_end - script_start, 2)

print("Script running time: ", time.strftime('%H:%M:%S', time.gmtime(time_delta)))